In [5]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import numpy as np
import scipy.sparse as scps
import scipy.sparse.linalg as ssl
import math

In [6]:
def maillage(n):
#
# Une discretisation possible d'une EDP elliptique sur le domaine ]0,1[ x ]0,1[
# Le carre [0,1]x[0,1] est maille uniquement avec des triangles; 
# Les conditions limites sont de type Dirichlet uniquement   => neumann  =[];
#
# Entrees :
# n : nombre de points par cote du care => Npts points de discretisation au
# total
#
# Sorties :
# coordinates : matrice a deux colonnes. Chaque ligne contient les 
# coordonnes 2D d'un des points de la discretisation. Ces sommets seront 
# identifies a l'indice de la ligne correspondante dans la matrice
# coordinates.
# elements3 : matrice a trois colonnes. Chaque ligne contient les indices 
# des sommets d'un element triangle, dans le sens antihoraire. 
# dirichlet : vecteur colonne des indices des sommets de la frontiere de
# Dirichlet.
# neumann : matrice a deux colonnes. Chaque ligne contient les indices 
# des deux sommets d'une arete de la frontiere de Neumann.
# (neumann est vide sur cet exemple)
#
##################################################################################
    h=1/(n-1)
    npoin       = n*n ; 
    nelem       = 2*(n-1)*(n-1) ;
    coordinates = np.zeros((npoin,2)); 
    elements3   = (np.zeros((nelem,3))).astype(int) ;
    neumann     = [];
    dirichlet=(np.zeros((4*n-4,1))).astype(int)
    # Coordonnees et connectivites :
    e = -1 ; 
    p = -1 ;
    x=np.zeros((n+1,1))
    x[n,0]=1.
    for l in range (n+1):
        x[l,0]=l*h
    for j in range (n):
            for i in range(n):
                p = p + 1  
                coordinates[p,0] = x[i,0]  
                coordinates[p,1] = x[j,0] 
                if ((i != n-1) & (j != n-1)):
                    p1 = p
                    p2 = p1 + 1 
                    p3 = p1 + n 
                    p4 = p2 + n 
                    e = e + 1 
                    elements3[e,0] = p1 
                    elements3[e,1] = p2 
                    elements3[e,2] = p3 
                    e = e + 1
                    elements3[e,0] = p4 
                    elements3[e,1] = p3 
                    elements3[e,2] = p2 
    #Liste des sommets de la frontiere de Dirichlet:
    p=-1
    for j in range(n):
        p=p+1
        dirichlet[p,0] = j  
    for j in range(n*2-1,n*(n-1),n):
        p=p+1
        dirichlet[p,0] = j 
    for j in range(n*n-1,n*n-n-1,-1):
        p=p+1
        dirichlet[p,0] = j 
    for j in range(n*n-2*n,n-1,-n):
        p=p+1
        dirichlet[p,0] = j 

    return coordinates, elements3,dirichlet, neumann

In [7]:
def show(coordinates,u):
#
# Fonction d'affichage de la solution u sur le maillage defini par
# elements3, coordinates.
#
# Entrees:
# elements3 : matrice a trois colonnes contenant les elements triangles
# de la discretisation, identifies par les indices de leurs trois
# sommets.
# coordinates : matrice a deux colonnes contenant les coordonnes 2D des
# points de la discretisation.
# u : vecteur colonne de longueur egale au nombre de lignes de
# coordinates contenant les valeurs de la solution a afficher aux
# points de la discretisation.
#
# Sorties : Aucune, mais la fonction doit s'afficher dans une figure.
##########################################################################
    ax= plt.figure().add_subplot(projection='3d')
    ax.plot_trisurf(coordinates[:,0],coordinates[:,1],u,linewidth=0.2,antialiased=True)
    plt.show()

**Partie I : maillage triangulaire et conditions de Dirichlet**

In [8]:
def raideurElem(coordinates, T) :
    
    T = [coordinates[T[i]] for i in range(3)]

    alpha = (T[1][0] - T[0][0]) * (T[2][1] - T[0][1]) - (T[1][1] - T[0][1]) * (T[2][0] - T[0][0])
    gradEta = [[T[(j + 1) % 3][1] - T[(j + 2) % 3][1], 
                T[(j + 2) % 3][0] - T[(j + 1) % 3][0]] for j in range(3)] /alpha
    gradEta = np.array(gradEta)
    aire = alpha/2
    M = aire * np.dot(gradEta, np.transpose(gradEta))

    return M


In [35]:
def raideur(n, coordinates, elements3) :
    A = np.zeros((n*n,n*n))
    for e in elements3 :
        M = raideurElem(coordinates, e)
        for i, elt_i in enumerate(e) :  #enumerate(e): pour trouver l'indice, puis la valeur
            for j, elt_j in enumerate(e) :
                A[elt_i, elt_j] += M[i][j]
    
    return A 


In [55]:
def secondMembreElem(n, coordinates, T, A, ud, f, frontiere) :
    bElem = np.zeros(3)
    Tcoordinates = [coordinates[T[i]] for i in range(3)]

    alpha = (Tcoordinates[1][0] - Tcoordinates[0][0]) * (Tcoordinates[2][1] - Tcoordinates[0][1]) - (Tcoordinates[1][1] - Tcoordinates[0][1]) * (Tcoordinates[2][0] - Tcoordinates[0][0])

    xg, yg = sum(Tcoordinates)/3


    for i in T :
        terme1 = alpha*f(xg,yg)/6
    #supposons g =0
    #supposons ud = 0

        terme2 = sum(ud * frontiere * A[i])
        bElem[i] = terme1 - terme2
        
    return bElem


In [58]:
def secondMembre(n, coordinates, A, ud, f, elements3) :
    frontiere = np.zeros(n*n)
    for i in range(n*n) : 
        if i <= n or i%n == 0 or i%n == n-1 or i >= (n-1) * n:
            frontiere[i] = 1

    b = np.zeros(n*n)
    for e in elements3 :
        bElem = secondMembreElem(n, coordinates, e, A, ud, f, frontiere)
        for i, elem in enumerate(e) :
            b[elem] += bElem[i]
    return b

In [59]:
n = 4
coordinates, elements3, dirichlet, neumann = maillage(n)
A = raideur(n, coordinates, elements3)
ud = np.zeros(n*n)
f = TODO
b = secondMembre(n, coordinates, A, ud, f, elements3)
    

NameError: name 'TODO' is not defined

**Partie II : maillage mixte et ajoût des conditions de Neumann**

In [53]:
############################# Maillage mixte ################
e3=np.array([[1,2,12],[2,3,12],[3,4,14],[4,5,14],[2,15,3],[3,15,4]]).astype(int)
e4=np.array([[0,1,12,11],[11,12,13,10],[12,3,14,13],[10,13,8,9],[13,14,7,8],[14,5,6,7]]).astype(int)
dds=np.array([2,15,4,6,7,8,9,10,11,0]).astype(int)
nns=np.array([[4,5],[5,6],[0,1],[1,2]]).astype(int)
ccs=np.array([[0.,0.],[0.33333333333333,0],[0.53333333333333,0.],
                      [0.66666666666667,0.33333333333333],[1.,0.47],[1,0.66666666666667],
                     [1.,1.],[0.66666666666667,1.],[0.33333333333333,1.], [0.,1.],
                     [0.,0.66666666666667],[0.,0.33333333333333],[0.33333333333333,0.33333333333333],
                     [0.33333333333333,0.66666666666667],[0.66666666666667,0.66666666666667],[1.,0.]])

**Compléments  :  un nouveau terme dans l'EDP**